In [1]:
#Import de librarii:

import pandas as pd
import re
import unicodedata


In [6]:
#Citirea dataset-urilor:
google_dataset = pd.read_csv('./datasets/google_dataset.csv', on_bad_lines='skip', low_memory=False)
website_dataset = pd.read_csv('./datasets/website_dataset.csv', on_bad_lines='skip')
facebook_dataset = pd.read_csv('./datasets/facebook_dataset.csv', on_bad_lines='skip')

#Inceperea curateniei in csv-uri prin redenumirea coloanelor + desfacerea lor
google_dataset_clean = google_dataset.rename(columns={
    'name': 'company_name', 'raw_address': 'address', 'category': 'categories', 'phone': 'phone_number'
})


website_dataset_clean = website_dataset['root_domain;domain_suffix;language;legal_name;main_city;main_country;main_region;phone;site_name;tld;s_category'].str.split(';', expand=True)
website_dataset_clean.columns = ['root_domain', 'domain_suffix', 'language', 'company_name', 'city', 'country_name', 'region_name', 'phone_number', 'site_name', 'tld', 'categories']


facebook_dataset_clean = facebook_dataset.rename(columns={
    'name': 'company_name', 'phone': 'phone_number', 'region_name': 'region_name', 'country_name': 'country_name'
})

#Eliminarea liniilor si a coloanelor ce contin 'NaN'(date lipsa) dintr-un DataFrame:
google_dataset_clean = google_dataset_clean.dropna(subset=['company_name'])
website_dataset_clean = website_dataset_clean.dropna(subset=['company_name'])
facebook_dataset_clean = facebook_dataset_clean.dropna(subset=['company_name'])

#Uniformizarea literelor:
google_dataset_clean['company_name'] = google_dataset_clean['company_name'].str.strip().str.lower()
website_dataset_clean['company_name'] = website_dataset_clean['company_name'].str.strip().str.lower()
facebook_dataset_clean['company_name'] = facebook_dataset_clean['company_name'].str.strip().str.lower()



In [9]:
#Salvarea dataseturilor curatate in fisiere csv separate, impreuna cu calea aferenta acestora(_path):
def save_clean_datasets(google_data, website_data, facebook_data, google_path, website_path, facebook_path):
    google_data.to_csv(google_path, index=False)
    website_data.to_csv(website_path, index=False)
    facebook_data.to_csv(facebook_path, index=False)


save_clean_datasets(google_dataset_clean, website_dataset_clean, facebook_dataset_clean, 
                    './datasets/google_cleaned.csv', './datasets/website_cleaned.csv', './datasets/facebook_cleaned.csv')

In [10]:
#Curatarea datelor din DataFrame eliminand spatiile suplimentare, inlocuind valorile nevalide si a caracterelor speciale:
def clean_data(df):

    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    
   
    df.replace({'#NAME?': np.nan, '""': np.nan, '"': '', 'nan': np.nan, '': 'N/A', ' ': 'N/A'}, inplace=True)
    
 
    df.dropna(how='all', inplace=True)
    
    return df


In [11]:
#Citeste dataseturile curatate, le imbina intr-un singur DataFrame si salveaza rezultatul intr-un fisier csv:
def read_merge_and_save_clean_datasets(google_path, website_path, facebook_path, output_path):

    google_data = pd.read_csv(google_path)
    website_data = pd.read_csv(website_path)
    facebook_data = pd.read_csv(facebook_path)
   
    merged_data = pd.merge(google_data, website_data, on='company_name', how='outer', suffixes=('_google', '_website'))
    merged_data = pd.merge(merged_data, facebook_data, on='company_name', how='outer', suffixes=('', '_facebook'))
    

    merged_data.to_csv(output_path, index=False)
    
    return merged_data


merged_dataset = read_merge_and_save_clean_datasets('./datasets/google_cleaned.csv', 
                                                    './datasets/website_cleaned.csv', 
                                                    './datasets/facebook_cleaned.csv', 
                                                    './datasets/companies_cleaned.csv')


print(merged_dataset.head())

C:\Users\ERIK FILIP\AppData\Local\Temp\ipykernel_8292\1529079150.py:15: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  google_data = pd.read_csv(google_path)


                                             address  \
0  Drama Theatre, Willenhall E-ACT Academy, Furze...   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4                                                NaN   

                categories_google city_google country_code  \
0  Theatres & Theatrical Services  willenhall           gb   
1                             NaN         NaN          NaN   
2                             NaN         NaN          NaN   
3                             NaN         NaN          NaN   
4                             NaN         NaN          NaN   

  country_name_google                                     company_name  \
0      united kingdom                                   !mpact theatre   
1                 NaN                             "\alpintech\"" llc."   
2                 NaN  "\flint hills community health center"" inc."

In [12]:
#La citire daca sunt linii incomplete sau incorecte, vor fi omise:
file_path = './datasets/companies_cleaned.csv'
df = pd.read_csv(file_path, on_bad_lines='skip',quoting=3)
print(df.head())

def remove_accents(text):
   #Normalizearea textului pentru a separa caracterele cu accente de cele de baza:
    text = unicodedata.normalize('NFKD', text)
    #Verifica daca un caracter este un semn de accent, si il elimina:
    text = "".join([c for c in text if not unicodedata.combining(c)])
    return text


def clean_text(text):
    #Verifica daca textul este NaN, si daca este il returneaza normal:
    if pd.isnull(text):
        return text
    #Eliminarea accentelor:
    text = remove_accents(text)

    #Eliminarea caracterelor neimprimabile din text:
    text = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)

    #Pastreaza doar literele, cifrele, spatiile și virgulele:
    text = re.sub(r'[^A-Za-z0-9\s,]', '', text)

    #Elimina semnele/literele dublate de mai jos:
    text = re.sub(r',+', ',', text)
    
    #Eliminam spatiile suplimentare si convertim textul la 'Title Case':
    return " ".join(text.split()).title()


def clean_phone(phone):
    
    #Verificam daca numarul de telefon este NaN:
    if pd.isnull(phone):
        return phone
    
    #Eliminarea tuturor caracterelor nenumerice si le trece sub forma '(123)456-7890':
    phone = ''.join([c for c in phone if c.isdigit()])
    if len(phone) == 10:
        return f"({phone[:3]}) {phone[3:6]}-{phone[6:]}"
    return phone

#Aplica functia 'clean' pe fiecare valoare din coloana, daca tipul de date al coloanei este 'object' (text). Dacă nu, lasă valorile neschimbate.
for column in df.columns:
    df[column] = df[column].apply(lambda x: clean_text(str(x)) if df[column].dtype == 'object' else x)

#Salarea Data Frame-ului curatat intr-un nou excel:
output_excel_path = './datasets/companies_cleaned_all_columns.xlsx'
df.to_excel(output_excel_path, index=False)



C:\Users\ERIK FILIP\AppData\Local\Temp\ipykernel_8292\2232309904.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,21,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, on_bad_lines='skip',quoting=3)


                                                                                                                                                     address  \
"Drama Theatre  Willenhall E-ACT Academy  Furzebank Way  Willenhall WV12 4BD  United Kingdom" Theatres & Theatrical Services                      willenhall   
NaN            NaN                       NaN            NaN                  NaN              """\alpintech\"""" llc."""                                 NaN   
                                                                                              """\flint hills community health center"""" inc...         NaN   
                                                                                              """\jeff home improvements"""" inc."""""""                 NaN   
                                                                                              """\major cargo service\ ltd."""""""                       NaN   

                                       

In [13]:
#Previzualizarea rezultatului:
print(df.columns)

Index(['address', 'categories_google', 'city_google', 'country_code',
       'country_name_google', 'company_name', 'phone_number_google',
       'phone_country_code', 'address.1', 'raw_phone', 'region_code',
       'region_name_google', 'text', 'zip_code', 'domain', 'root_domain',
       'domain_suffix', 'language', 'city_website', 'country_name_website',
       'region_name_website', 'phone_number_website', 'site_name', 'tld',
       'categories_website', 'domain_facebook', 'address_facebook',
       'categories', 'city', 'country_code_facebook', 'country_name',
       'description', 'email', 'link', 'page_type', 'phone_number',
       'phone_country_code_facebook', 'region_code_facebook', 'region_name',
       'zip_code_facebook'],
      dtype='object')


In [14]:
#Transformare din fisier .xlsx in .csv:
file_path = './datasets/companies_cleaned_all_columns.xlsx'
df = pd.read_excel(file_path)


output_csv_path = './datasets/companies_cleaned_all_columns.csv'
df.to_csv(output_csv_path, index=False)
print(f"Fisierul CSV a fost salvat la {output_csv_path}")

Fisierul CSV a fost salvat la ./datasets/companies_cleaned_all_columns.csv


In [15]:
#Verificare:
print(df.columns)

Index(['address', 'categories_google', 'city_google', 'country_code',
       'country_name_google', 'company_name', 'phone_number_google',
       'phone_country_code', 'address.1', 'raw_phone', 'region_code',
       'region_name_google', 'text', 'zip_code', 'domain', 'root_domain',
       'domain_suffix', 'language', 'city_website', 'country_name_website',
       'region_name_website', 'phone_number_website', 'site_name', 'tld',
       'categories_website', 'domain_facebook', 'address_facebook',
       'categories', 'city', 'country_code_facebook', 'country_name',
       'description', 'email', 'link', 'page_type', 'phone_number',
       'phone_country_code_facebook', 'region_code_facebook', 'region_name',
       'zip_code_facebook'],
      dtype='object')


In [16]:
import pandas as pd

#Citeste fisierul original excel cu toate coloanele:
file_path = './datasets/companies_cleaned_all_columns.xlsx'
df = pd.read_excel(file_path)

#Datorita RAM-ilor de care am dispus am selectat un nr finit de coloane :)) (SCUZE BTW)
df = df.head(100)

#Selecteaza coloanele relevante pentru Google, Facebook și Website:
columns_google = ['categories_google', 'address.1', 'phone_number_google', 'company_name']
columns_facebook = ['categories', 'address_facebook', 'phone_number', 'company_name']
columns_website = ['categories_website', 'phone_number_website', 'company_name', 'country_name_website', 'region_name_website']

#Selecteaza coloanele din fiecare sursa:
df_google = df[columns_google].rename(columns={
    'categories_google': 'Category',
    'address.1': 'Address',
    'phone_number_google': 'Phone',
    'company_name': 'Company'
})

df_facebook = df[columns_facebook].rename(columns={
    'categories': 'Category',
    'address_facebook': 'Address',
    'phone_number': 'Phone',
    'company_name': 'Company'
})

df_website = df[columns_website].rename(columns={
    'categories_website': 'Category',
    'phone_number_website': 'Phone',
    'company_name': 'Company',
    'country_name_website': 'Country',
    'region_name_website': 'Region'
})

# Combina Address-ul pentru Website din tara si regiune:
df_website['Address'] = df_website['Country'] + ', ' + df_website['Region']

#Selecteaza doar coloanele finale de interes din Website
df_website = df_website[['Category', 'Phone', 'Address', 'Company']]

#Face merge intre Google, Facebook si Website pe baza numelui companiei:
df_merged = pd.merge(df_google, df_facebook, on='Company', how='outer', suffixes=('_google', '_facebook'))
df_merged = pd.merge(df_merged, df_website, on='Company', how='outer')

#Unificam valorile din Google, Facebook si Website pentru fiecare coloana:
df_final = pd.DataFrame()
df_final['Company'] = df_merged['Company']

#Pentru Category, Address si Phone, alegem prima valoare disponibila din cele trei surse (Google, Facebook, Website):
df_final['Category'] = df_merged['Category_google'].combine_first(df_merged['Category_facebook']).combine_first(df_merged['Category'])

df_final['Address'] = df_merged['Address_google'].combine_first(df_merged['Address_facebook']).combine_first(df_merged['Address'])

df_final['Phone'] = df_merged['Phone_google'].combine_first(df_merged['Phone_facebook']).combine_first(df_merged['Phone'])

#Salvam intr-un nou fisier csv si excel:
output_csv_path = './datasets/merged_companies_first_100.csv'
df_final.to_csv(output_csv_path, index=False)

output_excel_path = './datasets/merged_companies_first_100.xlsx'
df_final.to_excel(output_excel_path, index=False)

print(f"Fisierul final a fost salvat la {output_csv_path} si {output_excel_path}")


Fisierul final a fost salvat la ./datasets/merged_companies_first_100.csv si ./datasets/merged_companies_first_100.xlsx
